In [1]:
#load packages
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd

In [2]:

# define functions

def danish_sentences_extraction(data, text_column):


    rows = range(len)
    sentences = {}

    for para in rows:
        #sentence_indices = dict[para]
        text_sentences = ast.literal_eval(data.loc[para][text_column])
        
        blame_sentence_dict = {}
        for indx in range(len(text_sentences)):
            blame_sentence = text_sentences[indx]
            blame_sentence_dict[indx] = blame_sentence
        
        
        sentences[para] = blame_sentence_dict

    return sentences
    


In [9]:
#Fist create inference dataset

df = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")

# Convert the column to datetime (specify format if necessary)
df["date"] = pd.to_datetime(df["date"], errors='coerce', dayfirst=True)

# Filter rows after January 1, 2000
filtered_df = df[df["date"] > "2000-01-01"]

#filtered_df.pop("Unnamed: 0")

filtered_df.head()

,Unnamed: 0,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
36314,36314,2000-11-01,1) Spørgsmål til ministrene.,2,Henning Grove,NaN,NaN,True,181,Til at besvare spørgsmål i spørgetimen i dag h...,DK-Folketing,DNK
36315,36315,2000-11-01,1) Spørgsmål til ministrene.,3,Frank Dahlgaard,UP,NaN,False,187,Ministeren er jo ikke bare minister for fødeva...,DK-Folketing,DNK
36316,36316,2000-11-01,1) Spørgsmål til ministrene.,4,Henrik Dam Kristensen,S,379.0,False,175,Først vil jeg gerne sige til hr. Frank Dahlgaa...,DK-Folketing,DNK
36317,36317,2000-11-01,1) Spørgsmål til ministrene.,5,Frank Dahlgaard,UP,NaN,False,173,"Ja, vi har en stor eksport, men ministeren er ...",DK-Folketing,DNK
36318,36318,2000-11-01,1) Spørgsmål til ministrene.,6,Henrik Dam Kristensen,S,379.0,False,138,"Hr. Frank Dahlgaard har jo fuldstændig ret i, ...",DK-Folketing,DNK


In [7]:
len(filtered_df)

293615

In [ ]:
filtered_df_sub = filtered_df[:10]



In [ ]:
all_sentences = danish_sentences_extraction(filtered_df_sub, 'segmented_text')

In [ ]:


with open("/work/MarkusLundsfrydJensen#1865/inference_data.json", mode='w', encoding='utf-8') as jsonfile:
    json.dump(filtered_df, jsonfile, indent=4)


In [ ]:
[
  {
    "paragraph": "2",
    "sentence_nr": "235",
    "text": "Regeringen vil også fortsætte sin offensive  miljøpolitik.",
    "speaker": "Poul Nyrup Rasmussen",
    "party": "S",
    "preceding_sentence": "Der indføres en belønningsordning for virksomheder, der gør  en særlig ekstraindsats ud over det, loven stiller krav om,  for at forbedre arbejdsmiljøet.",
    "succeeding_sent": "Det er vores næste store felt.",
    "current_speaker_in_government": true,
    "parties_in_government": [
      "S",
      "RV"
    ],
    "date": "1997-10-07 00:00:00",
    "label": 1
  },

In [ ]:
import ipywidgets
#tokenizer = AutoModel.from_pretrained("/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model")
model = AutoModel.from_pretrained("/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model")


pytorch_model.bin:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Loading adapter weights from /work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model led to unexpected keys not found in the model: model.layers.0.attn.Wqkv.lora_A.default.weight, model.layers.0.attn.Wqkv.lora_B.default.weight, model.layers.1.attn.Wqkv.lora_A.default.weight, model.layers.1.attn.Wqkv.lora_B.default.weight, model.layers.10.attn.Wqkv.lora_A.default.weight, model.layers.10.attn.Wqkv.lora_B.default.weight, model.layers.11.attn.Wqkv.lora_A.default.weight, model.layers.11.attn.Wqkv.lora_B.default.weight, model.layers.12.attn.Wqkv.lora_A.default.weight, model.layers.12.attn.Wqkv.lora_B.default.weight, model.layers.13.attn.Wqkv.lora_A.default.weight, model.layers.13.attn.Wqkv.lora_B.default.weight, model.layers.14.attn.Wqkv.lora_A.default.weight, model.layers.14.attn.Wqkv.lora_B.default.weight, model.layers.15.attn.Wqkv.lora_A.default.weight, model.layers.15.attn.Wqkv.lora_B.default.weight, model.layers.16.attn.Wqkv.lora_A.default.weight, model.laye

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("jhu-clsp/mmBERT-base")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
import numpy as np

# Configuration
MODEL_PATH = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model"
BASE_MODEL = "jhu-clsp/mmBERT-base" 
MAX_LENGTH = 512  # ModernBERT supports up to 8192
NUM_LABELS = 2  # Binary classification

def load_model_with_lora(model_path, base_model_name, num_labels=2):
    """Load the fine-tuned ModernBERT model with LoRA adapters."""
    print(f"Loading model from {model_path}...")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    # Load base model with classification head
    print(f"Loading base model: {base_model_name}")
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=num_labels,
        trust_remote_code=True
    )
    
    # Load LoRA adapters
    print("Loading LoRA adapters...")
    model = PeftModel.from_pretrained(model, model_path)
    
    # Merge LoRA weights with base model for faster inference (optional)
    print("Merging LoRA weights...")
    model = model.merge_and_unload()
    
    # Set model to evaluation mode
    model.eval()
    
    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    print(f"Model loaded successfully on {device}")
    return model, tokenizer, device

def predict(text, model, tokenizer, device, max_length=512):
    """Make a prediction on a single text input."""
    # Tokenize input
    inputs = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    
    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    return predicted_class, confidence, probabilities[0].cpu().numpy()

def predict_batch(texts, model, tokenizer, device, max_length=512):
    """Make predictions on a batch of texts."""
    # Tokenize inputs
    inputs = tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    
    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_classes = torch.argmax(probabilities, dim=1).cpu().numpy()
        confidences = probabilities.max(dim=1).values.cpu().numpy()
    
    return predicted_classes, confidences, probabilities.cpu().numpy()

def main():
    # Load model and tokenizer
    model, tokenizer, device = load_model_with_lora(
        MODEL_PATH, 
        BASE_MODEL, 
        NUM_LABELS
    )
    
    # Test examples
    test_texts = [
        "Goddad min kære ven, jeg kan virkelig godt lide dig.",
        "Jeg vil blot sige til fødevareministeren, at han er grim .",
        "Dernædt vil jeg blot understrege de tydelige beviser på immigraternes tydelige skyld i nedgangen i økonomien."
    ]
    
    print("\n" + "="*70)
    print("Testing ModernBERT Binary Classifier with LoRA")
    print("="*70 + "\n")
    
    # Single predictions
    print("SINGLE PREDICTIONS:")
    print("-" * 70)
    for i, text in enumerate(test_texts, 1):
        print(f"\nTest {i}:")
        print(f"Input: {text}")
        
        predicted_class, confidence, probs = predict(
            text, model, tokenizer, device, MAX_LENGTH
        )
        
        print(f"Predicted Class: {predicted_class}")
        print(f"Confidence: {confidence:.4f}")
        print(f"Probabilities: [Class 0: {probs[0]:.4f}, Class 1: {probs[1]:.4f}]")
    
    # Batch prediction example
    print("\n" + "="*70)
    print("BATCH PREDICTION:")
    print("-" * 70)
    classes, confidences, probs = predict_batch(
        test_texts, model, tokenizer, device, MAX_LENGTH
    )
    
    for i, (text, cls, conf) in enumerate(zip(test_texts, classes, confidences), 1):
        print(f"\nText {i}: {text}")
        print(f"Class: {cls}, Confidence: {conf:.4f}")

if __name__ == "__main__":
    main()

Loading model from /work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model...
Loading base model: jhu-clsp/mmBERT-base


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading LoRA adapters...
Merging LoRA weights...
Model loaded successfully on cpu

Testing ModernBERT Binary Classifier with LoRA

SINGLE PREDICTIONS:
----------------------------------------------------------------------

Test 1:
Input: Goddad min kære ven, jeg kan virkelig godt lide dig.
Predicted Class: 0
Confidence: 0.7502
Probabilities: [Class 0: 0.7502, Class 1: 0.2498]

Test 2:
Input: Jeg vil blot sige til fødevareministeren, at han er grim .
Predicted Class: 0
Confidence: 0.5292
Probabilities: [Class 0: 0.5292, Class 1: 0.4708]

Test 3:
Input: Dernædt vil jeg blot understrege de tydelige beviser på immigraternes tydelige skyld i nedgangen i økonomien.
Predicted Class: 1
Confidence: 0.8764
Probabilities: [Class 0: 0.1236, Class 1: 0.8764]

BATCH PREDICTION:
----------------------------------------------------------------------

Text 1: Goddad min kære ven, jeg kan virkelig godt lide dig.
Class: 0, Confidence: 0.7502

Text 2: Jeg vil blot sige til fødevareministeren, at han er gr

In [ ]:
#OBS need to be made usable
'''class BlameDetectorDa(object):

    def __init__(self, model_path, max_length, batch_size = None):

        self.model_path = model_path
        self.max_length = max_length
        self.batch_size = batch_size

        self.model_initialization()

        return

    def model_initialization(self):
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

        self.model.eval()
            
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        print(f"Model loaded successfully on {self.device}")

        return

    def predict(self):
        """Make a prediction on a single text input."""
        # Tokenize input
        inputs = self.tokenizer(
            self.text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def run_prediction(self, text):

        self.text = text
        predicted_class, confidence, probs = self.predict()
            
        return predicted_class, confidence
'''


In [ ]:
example = "Goddad min kære ven, jeg kan virkelig godt lide dig."

BD = BlameDetectorDa()
BD.run_prediction(text = example)

In [35]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from typing import Union, List, Tuple

class BlameDetectorDa(object):
    """
    Danish Blame Detection classifier with support for single and batch predictions.
    """

    def __init__(self, model_path: str, max_length: int = 512, batch_size: int = None, base_model_name = "jhu-clsp/mmBERT-base", num_labels = 2):
        """
        Initialize the BlameDetectorDa classifier.
        
        Args:
            model_path: Path to the pretrained model
            max_length: Maximum sequence length for tokenization
            batch_size: Batch size for batch predictions (None for single predictions)
        """
        self.model_path = model_path
        self.max_length = max_length
        self.batch_size = batch_size

        self.base_model_name = base_model_name #delete when model is ready
        self.num_labels = num_labels #delete when model is ready

        self.model_initialization_prelim() #change when model is ready

    def model_initialization(self):
        """Initialize the model, tokenizer, and device."""
        print(f"Loading model from {self.model_path}...")
        
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_path,
            trust_remote_code=True  # Add this for ModernBERT
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

        self.model.eval()
            
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)  # Fixed: was 'device', should be 'self.device'

        print(f"Model loaded successfully on {self.device}")

    def model_initialization_prelim(self):
        print(f"Loading model from {self.model_path}...")
    
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.base_model_name)
        
        # Load base model with classification head
        print(f"Loading base model: {self.base_model_name}")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.base_model_name,
            num_labels=self.num_labels,
            trust_remote_code=True
        )
        
        # Load LoRA adapters
        print("Loading LoRA adapters...")
        self.model = PeftModel.from_pretrained(self.model, self.model_path)
        
        # Merge LoRA weights with base model for faster inference (optional)
        print("Merging LoRA weights...")
        self.model = self.model.merge_and_unload()
        
        # Set model to evaluation mode
        self.model.eval()
        
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        
        print(f"Model loaded successfully on {self.device}")
        return
        

    def predict_single(self, text: str) -> Tuple[int, float, np.ndarray]:
        """
        Make a prediction on a single text input.
        
        Args:
            text: Input text to classify
            
        Returns:
            Tuple of (predicted_class, confidence, probabilities)
        """
        # Tokenize input
        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def predict_batch(self, texts: List[str]) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Make predictions on a batch of text inputs.
        
        Args:
            texts: List of input texts to classify
            
        Returns:
            Tuple of (predicted_classes, confidences, all_probabilities)
        """
        # Tokenize all inputs
        inputs = self.tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Make predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_classes = torch.argmax(probabilities, dim=1).cpu().numpy()
            confidences = probabilities.max(dim=1).values.cpu().numpy()
            all_probs = probabilities.cpu().numpy()
        
        return predicted_classes, confidences, all_probs

    def run_prediction(
        self, 
        text: Union[str, List[str]]
    ) -> Union[Tuple[int, float], Tuple[np.ndarray, np.ndarray]]:
        """
        Run prediction on single text or batch of texts.
        
        Args:
            text: Single text string or list of text strings
            
        Returns:
            For single text: (predicted_class, confidence)
            For batch: (predicted_classes, confidences)
        """
        # Check if input is a list (batch) or single text
        if isinstance(text, list):
            # Batch prediction
            if self.batch_size is not None and len(text) > self.batch_size:
                # Process in batches
                all_classes = []
                all_confidences = []
                
                for i in range(0, len(text), self.batch_size):
                    batch = text[i:i + self.batch_size]
                    classes, confidences, _ = self.predict_batch(batch)
                    all_classes.extend(classes)
                    all_confidences.extend(confidences)
                
                return np.array(all_classes), np.array(all_confidences)
            else:
                # Single batch prediction
                predicted_classes, confidences, _ = self.predict_batch(text)
                return predicted_classes, confidences
        else:
            # Single text prediction
            predicted_class, confidence, _ = self.predict_single(text)
            return predicted_class, confidence


'''# Example usage

# Initialize detector
MODEL_PATH = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_merged"
detector = BlameDetectorDa(
    model_path=MODEL_PATH,
    max_length=512,
    batch_size=8  # Process 8 texts at a time
)

print("Single Prediction Test")


# Single prediction
single_text = "Dette er en test sætning."
pred_class, confidence = detector.run_prediction(single_text)
print(f"\nText: {single_text}")
print(f"Predicted Class: {pred_class}")
print(f"Confidence: {confidence:.4f}")

print("Batch Prediction Test")

# Batch prediction
batch_texts = [
    "Første test sætning.",
    "Anden test sætning.",
    "Tredje test sætning.",
    "Fjerde test sætning.",
]

pred_classes, confidences = detector.run_prediction(batch_texts)

print("\nBatch Results:")
for i, (text, cls, conf) in enumerate(zip(batch_texts, pred_classes, confidences), 1):
    print(f"\n{i}. Text: {text}")
    print(f"   Class: {cls}, Confidence: {conf:.4f}")'''

'# Example usage\n\n# Initialize detector\nMODEL_PATH = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_merged"\ndetector = BlameDetectorDa(\n    model_path=MODEL_PATH,\n    max_length=512,\n    batch_size=8  # Process 8 texts at a time\n)\n\nprint("Single Prediction Test")\n\n\n# Single prediction\nsingle_text = "Dette er en test sætning."\npred_class, confidence = detector.run_prediction(single_text)\nprint(f"\nText: {single_text}")\nprint(f"Predicted Class: {pred_class}")\nprint(f"Confidence: {confidence:.4f}")\n\nprint("Batch Prediction Test")\n\n# Batch prediction\nbatch_texts = [\n    "Første test sætning.",\n    "Anden test sætning.",\n    "Tredje test sætning.",\n    "Fjerde test sætning.",\n]\n\npred_classes, confidences = detector.run_prediction(batch_texts)\n\nprint("\nBatch Results:")\nfor i, (text, cls, conf) in enumerate(zip(batch_texts, pred_classes, confidences), 1):\n    print(f"\n{i}. Text: {text}")\n    print(f"   Class: {cls}, Confide

In [55]:
import json
from tqdm import tqdm  # for progress bar (optional)

MODEL_PATH = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model"
detector = BlameDetectorDa(
    model_path=MODEL_PATH,
    max_length=1024,
    batch_size=2  # Process 2 texts at a time
)


Loading model from /work/MarkusLundsfrydJensen#1865/Bachelor_project/output/mmBERT/template_3_4_5_model...
Loading base model: jhu-clsp/mmBERT-base


Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading LoRA adapters...
Merging LoRA weights...
Model loaded successfully on cpu


In [56]:
# ---- 1. Load JSON dataset ----
import json
json_path = "/work/MarkusLundsfrydJensen#1865/Training_data/cleaned_training_data_3_4_5_temps.json"
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)



In [117]:


# ---- 2. Extract texts ----
texts = [item["text"] for item in new_data if "text" in item]

# ---- 3. Run predictions in batches ----
pred_classes, confidences = detector.run_prediction(texts)


# ---- 4. Attach predictions back to data ----
for item, cls, conf in zip(new_data, pred_classes, confidences):
    item["predicted_class"] = int(cls)
    item["confidence"] = float(conf)


#save data



In [123]:
for entry in new_data:
    if entry["label"] == 1:
        print(entry["predicted_class"])
        print(entry["confidence"])

0
0.5366607308387756
0
0.7602487802505493
